In [ ]:
import wec_as_multiport as wam
import wecopttool as wot
import capytaine as cpy
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter, NullFormatter
import pandas as pd

gfx_path = os.path.join('..','gfx')
base_name = 'wec_as_multiport_'

In [ ]:
f1 = 0.025
nfreq = 60
freq = wot.frequency(f1, nfreq, False)  # False -> no zero frequency

wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5  # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")

In [ ]:
bem_data_fname = 'wec_as_multiport.nc'

if os.path.isfile(bem_data_fname):
    bem_data = wot.read_netcdf(bem_data_fname)
else:
    bem_data = wot.run_bem(fb, freq)
    bem_data = bem_data.assign_coords(
        freq=("omega", bem_data['omega'].values/2/np.pi))
    bem_data['freq'].attrs['long_name'] = 'Frequency'
    bem_data['freq'].attrs['units'] = 'Hz'
    bem_data['excitation_force'] = bem_data['diffraction_force'] + bem_data['Froude_Krylov_force']
    bem_data = wot.add_linear_friction(bem_data)
    wot.write_netcdf(bem_data_fname, bem_data)

In [ ]:
wec = wam.WEC(omega=bem_data['omega'].values,
            N=12.4666,
            Kt=6.1745,
            Rw=0.5,
            Lw=0,
            Jd=2,
            Bd=1,
            Kd=0,
            Zi=np.squeeze(wot.hydrodynamic_impedance(bem_data)).values,
            Hexc=np.squeeze(bem_data['excitation_force'].values))

In [ ]:
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zi)), color='k')
ax[0].set_ylabel('$|Z_i|$ [dB]')

ax[1].plot(wec.freq, np.angle(wec.Zi), color='k')
ax[1].set_ylabel('$\\angle{Z_i}$ [rad]')

ax[-1].set_xlabel('Frequency [Hz]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='major', axis='x')
    axi.set_title('')
    axi.label_outer()

In [ ]:
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

ax[0].plot(wec.freq, np.real(wec.Zi), color='k')
ax[0].set_ylabel('$\\Re\{Z_i\}$ [XX]')

ax[1].plot(wec.freq, np.imag(wec.Zi), color='k')
ax[1].set_ylabel('$\\Im\{Z_i\}$ [XX]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    
ax[-1].set_xlabel('Frequency [Hz]')

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(wec.freq, np.abs(wec.Hexc), color='k')
ax[1].plot(wec.freq, np.angle(wec.Hexc), color='k')

ax[0].set_ylabel('$| H_{exc} | $ [N/m]')
ax[1].set_ylabel('$\\angle{H_{exc}}$ [rad]')
ax[-1].set_xlabel('Frequency [Hz]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()

# Contoller design

## Controller impedances

In [ ]:
fig, ax = plt.subplots(nrows=3,
                       sharex=True)

ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zl_opt_mech)),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zl_opt)),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[1].plot(wec.freq, np.angle(wec.Zl_opt_mech),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[1].plot(wec.freq, np.angle(wec.Zl_opt),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq, wec.transducer_power_gain(Zl=wec.Zl_opt), 
        ls='-', 
        color = 'k', 
        label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq,wec.transducer_power_gain(Zl=wec.Zl_opt_mech), 
        ls='--', 
        color = 'k', 
        label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    axi.xaxis.set_minor_formatter(NullFormatter())
    axi.set_xlim([1e-1,1])
    
ax[2].set_yscale('linear')
ax[2].set_ylim([-1,1])
    
ax[0].legend()
ax[-1].set_xlabel('Frequency [Hz]')

ax[0].set_ylabel('$|Z|$ [dB]')
ax[1].set_ylabel('$\\angle{Z}$ [rad]')
ax[2].set_ylabel('Trans. gain')

fig.savefig(os.path.join(gfx_path,base_name + 'load_impedance_for_mech_power_Bode.pdf'))

In [ ]:
fig, ax = plt.subplots(nrows=3,
                       sharex=True)

ax[0].plot(wec.freq, np.real(np.abs(wec.Zl_opt_mech)),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[0].plot(wec.freq, np.real(np.abs(wec.Zl_opt)),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[1].plot(wec.freq, np.imag(wec.Zl_opt_mech),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[1].plot(wec.freq, np.imag(wec.Zl_opt),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq, wec.transducer_power_gain(Zl=wec.Zl_opt), 
        ls='-', 
        color = 'k', 
        label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq,wec.transducer_power_gain(Zl=wec.Zl_opt_mech), 
        ls='--', 
        color = 'k', 
        label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    axi.xaxis.set_minor_formatter(NullFormatter())
    axi.set_xlim([1e-1,1])
    
ax[2].set_yscale('linear')
ax[2].set_ylim([-1,1])
    
ax[0].legend()
ax[-1].set_xlabel('Frequency [Hz]')

ax[0].set_ylabel('$\\Re \{ Z \}$ [XX]')
ax[1].set_ylabel('$\\Im \{ Z \}$ [XX]')
ax[2].set_ylabel('Trans. gain')

fig.savefig(os.path.join(gfx_path,base_name + 'load_impedance_for_mech_power_real_imag.pdf'))

## Power output

In [ ]:
waves = wot.waves.regular_wave(f1=wec.f1, nfreq=wec.nfreq, freq=0.4, amplitude=0.25)
Fexc = wec.Fexc(waves=waves.squeeze().values)
Fth = wec.F_Thevenin(Fexc=Fexc)

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(np.abs(Fth), label='Thevenin')
ax[0].plot(np.abs(Fexc), label='Regular')
ax[0].set_ylabel('Excitation')

ax[1].plot(np.real(wec.Z_Thevenin))
ax[1].plot(np.real(wec.Zi))
ax[1].set_ylabel('Real impedance')

ax[2].plot(wec.max_active_power(Fexc=Fexc))
ax[2].plot(wec.max_active_power_mech(Fexc=Fexc))
ax[2].set_ylabel('Max power')

ax[0].legend()

In [ ]:
# wot.waves.jonswap_spectrum(freq=wec.freq, fp=0.4, hs=1, gamma=1)
# wot.waves.long_crested_wave()

fp_list = [0.4, 0.5, 0.6]
pow_list = []

# wec.Rw = 0?

for fp in fp_list:
    # efth = wot.waves.omnidirectional_spectrum(wec.f1, 
    #                                         wec.nfrq, 
    #                                         lambda f: wot.waves.pierson_moskowitz_spectrum(f, fp, 0.2), 
    #                                         "Pierson-Moskowitz")
    # waves = wot.waves.long_crested_wave(efth, 1)
    waves = wot.waves.regular_wave(f1=wec.f1, nfreq=wec.nfreq, freq=fp, amplitude=0.25)
    Fexc = wec.Fexc(waves=waves.squeeze().values)
    pow = np.sum(wec.active_power(Fexc=Fexc, Zl=wec.Zl_opt))
    pow_list.append(pow)
    # print(pow)
    
df = pd.DataFrame(pow_list, fp_list, 
                  columns=['Elec. power [W]'])
df
# print(df.transpose().to_latex(float_format='%.1f'))

# Compare different drive trains

## Kd

In [ ]:
Kd_list = [0, -50, -100]

wec_compare_Kd = []

for Kd in Kd_list:
    wec_tmp = wec.copy()
    wec_tmp.Kd = Kd
    wec_compare_Kd.append(wec_tmp)

fig, ax = plt.subplots(nrows=2,
                       ncols=2,
                       sharex=True)

colors = plt.cm.Reds(np.linspace(0.4, 0.75, len(wec_compare_Kd)))
mask = (wec.freq >= 0.1) & (wec.freq <= 1)

for wec1, color in zip(wec_compare_Kd, colors):
    ax[0,0].plot(wec1.freq[mask], np.real(wec1.Zin()[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(k_d=$' + f'{wec1.Kd})')
    ax[1,0].plot(wec1.freq[mask], np.imag(wec1.Zin()[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(k_d=$' + f'{wec1.Kd})')
    
    ax[0,1].plot(wec1.freq[mask], np.real(wec1.Zout[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(k_d=$' + f'{wec1.Kd})')
    ax[1,1].plot(wec1.freq[mask], np.imag(wec1.Zout[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(k_d=$' + f'{wec1.Kd})')
    

ax[0,0].plot(wec.freq[mask], np.real(np.conj(wec.Zi[mask])),
           color='k',
           ls='--',
           label='$Z_i^*$')
ax[1,0].plot(wec.freq[mask], np.imag(np.conj(wec.Zi[mask])),
           color='k',
           ls='--',
           label='$Z_i^*$')

for axi in ax.flatten():
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    # axi.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
    axi.xaxis.set_minor_formatter(NullFormatter())

ax[0,0].set_ylabel('$\\Re\{Z\}$ [XX]')
ax[1,0].set_ylabel('$\\Im\{Z\}$ [XX]')

ax[1,0].legend(ncol=2, fontsize='x-small')
# ax[-1].set_xlabel('Frequency [Hz]')

ax[0][0].set_title('PTO input')
ax[0][1].set_title('PTO output')
fig.text(0.5, 0.02, 'Frequency [Hz]', ha='center')

# fig.savefig(os.path.join(gfx_path,base_name + 'in_and_out_impedances_spring.pdf'))

## Jd

In [ ]:
Jd_list = [2, 10, 20]

wec_compare_Jd = []

for Jd in Jd_list:
    wec_tmp = wec.copy()
    wec_tmp.Jd = Jd
    wec_compare_Jd.append(wec_tmp)

fig, ax = plt.subplots(nrows=2,
                       ncols=2,
                       sharex=True)

colors = plt.cm.Blues(np.linspace(0.4, 0.75, len(wec_compare_Jd)))
mask = (wec.freq >= 0.1) & (wec.freq <= 1)

for wec1, color in zip(wec_compare_Jd, colors):
    ax[0,0].plot(wec1.freq[mask], np.real(wec1.Zin()[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(J_d=$' + f'{wec1.Jd})')
    ax[1,0].plot(wec1.freq[mask], np.imag(wec1.Zin()[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(J_d=$' + f'{wec1.Jd})')
    
    ax[0,1].plot(wec1.freq[mask], np.real(wec1.Zout[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(J_d=$' + f'{wec1.Jd})')
    ax[1,1].plot(wec1.freq[mask], np.imag(wec1.Zout[mask]),
               color=color,
               label='$Z_{\mathrm{in}}(J_d=$' + f'{wec1.Jd})')
    

ax[0,0].plot(wec.freq[mask], np.real(np.conj(wec.Zi[mask])),
           color='k',
           ls='--',
           label='$Z_i^*$')
ax[1,0].plot(wec.freq[mask], np.imag(np.conj(wec.Zi[mask])),
           color='k',
           ls='--',
           label='$Z_i^*$')

for axi in ax.flatten():
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    # axi.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
    axi.xaxis.set_minor_formatter(NullFormatter())

ax[0,0].set_ylabel('$\\Re\{Z\}$ [XX]')
ax[1,0].set_ylabel('$\\Im\{Z\}$ [XX]')

ax[1,0].legend(ncol=2, fontsize='x-small')
# ax[-1].set_xlabel('Frequency [Hz]')

ax[0][0].set_title('PTO input')
ax[0][1].set_title('PTO output')
fig.text(0.5, 0.02, 'Frequency [Hz]', ha='center')

# fig.savefig(os.path.join(gfx_path,base_name + 'in_and_out_impedances_inertia.pdf'))